### Package installation

In [ ]:
pip install psycopg2

### Package call

In [2]:
import psycopg2
import pandas as pd
import psycopg2.extras

### Extract

***Read from excel file***

In [32]:
Trans_dim = pd.read_excel(open('e-commerece_data.xlsx','rb'), sheet_name=1, engine='openpyxl')
Trans_dim

,payment_key,trans_type,bank_name
0,P001,cash,None
1,P002,card,AB Bank Limited
2,P003,card,Bangladesh Commerce Bank Limited
3,P004,card,Bank Asia Limited
4,P005,card,BRAC Bank Limited
5,P006,card,Citizens Bank Limited
6,P007,card,City Bank Limited
7,P008,card,Community Bank Bangladesh Limited
8,P009,card,Dhaka Bank Limited
9,P010,card,Dutch-Bangla Bank Limited


In [149]:
Item_dim = pd.read_excel(open('e-commerece_data.xlsx','rb'), sheet_name=2, engine='openpyxl')
Item_dim

,item_key,item_name,desc,unit_price,man_country,supplier,unit
0,I00001,A&W Root Beer - 12 oz cans,a. Beverage - Soda,11.50,Netherlands,Bolsius Boxmeer,cans
1,I00002,A&W Root Beer Diet - 12 oz cans,a. Beverage - Soda,6.75,poland,CHROMADURLIN S.A.S,cans
2,I00003,Barq's Root Beer - 12 oz cans,a. Beverage - Soda,6.75,Bangladesh,DENIMACH LTD,cans
3,I00004,Cherry Coke 12oz,a. Beverage - Soda,6.75,Netherlands,Bolsius Boxmeer,cans
4,I00005,Cherry Coke Zero 12 pack,a. Beverage - Soda,6.75,Finland,HARDFORD AB,cans
...,...,...,...,...,...,...,...
259,I00260,Napkins Square Lunch,Kitchen Supplies,15.00,Cambodia,NINGBO SEDUNO IMP & EXP CO.LTD,ct
260,I00261,POM 2 ply paper towels,Kitchen Supplies,33.00,Lithuania,BIGSO AB,ct
261,I00262,Strong Everyday Napkins,Kitchen Supplies,14.00,Netherlands,Bolsius Boxmeer,ct
262,I00263,Advil 2 pill packets,Medicine,14.00,Lithuania,BIGSO AB,Ct


In [150]:
Item_dim.isnull().sum()

item_key       0
item_name      0
desc           0
unit_price     0
man_country    0
supplier       0
unit           1
dtype: int64

In [151]:
Item_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   item_key     264 non-null    object 
 1   item_name    264 non-null    object 
 2   desc         264 non-null    object 
 3   unit_price   264 non-null    float64
 4   man_country  264 non-null    object 
 5   supplier     264 non-null    object 
 6   unit         263 non-null    object 
dtypes: float64(1), object(6)
memory usage: 14.6+ KB


In [42]:
Coustomer_dim = pd.read_excel(open('e-commerece_data.xlsx','rb'), sheet_name=3, engine='openpyxl')
Coustomer_dim

,coustomer_key,name,contact_no,nid
0,C000001,sumit,8801920345851,7505075708899
1,C000002,tammanne,8801817069329,1977731324842
2,C000003,kailash kumar,8801663795774,3769494056318
3,C000004,bhagwati prasad,8801533627961,9378834712725
4,C000005,ajay,8801943715786,3540815556323
...,...,...,...,...
9186,C009187,smt radha,8801563029753,1057824928189
9187,C009188,tulsi sharma,8801794982266,7970880905205
9188,C009189,madhubala,8801890032435,3101930298150
9189,C009190,pintu devi,8801563290767,7234328323528


In [132]:
Coustomer_dim.isnull().sum()

coustomer_key     0
name             27
contact_no        0
nid               0
dtype: int64

In [45]:
Time_dim = pd.read_excel(open('e-commerece_data.xlsx','rb'), sheet_name=4, engine='openpyxl')
Time_dim

,time_key,date,hour,day,week,month,quarter,year
0,T00001,20-05-2017 14:56,14,20,3rd Week,5,Q2,2017
1,T00002,30-01-2015 22:14,22,30,4th Week,1,Q1,2015
2,T00003,14-03-2020 02:34,2,14,2nd Week,3,Q1,2020
3,T00004,27-04-2018 12:19,12,27,4th Week,4,Q2,2018
4,T00005,14-04-2018 10:43,10,14,2nd Week,4,Q2,2018
...,...,...,...,...,...,...,...,...
99994,T099995,04-01-2015 17:22,17,4,1st Week,1,Q1,2015
99995,T099996,17-01-2015 21:09,21,17,3rd Week,1,Q1,2015
99996,T099997,23-01-2020 14:42,14,23,4th Week,1,Q1,2020
99997,T099998,31-10-2015 13:52,13,31,4th Week,10,Q4,2015


In [35]:
Store_dim = pd.read_excel(open('e-commerece_data.xlsx','rb'), sheet_name=5, engine='openpyxl')
Store_dim

,store_key,division,district,upazila
0,S0001,SYLHET,HABIGANJ,AJMIRIGANJ
1,S0002,SYLHET,HABIGANJ,BAHUBAL
2,S0003,SYLHET,HABIGANJ,BANIACHONG
3,S0004,SYLHET,HABIGANJ,CHUNARUGHAT
4,S0005,SYLHET,HABIGANJ,HABIGANJ SADAR
...,...,...,...,...
721,S00722,BARISAL,BARISAL,HIZLA
722,S00723,BARISAL,BARISAL,BARISAL SADAR (KOTWALI)
723,S00724,BARISAL,BARISAL,MEHENDIGANJ
724,S00725,BARISAL,BARISAL,MULADI


In [134]:
Store_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726 entries, 0 to 725
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_key  726 non-null    object
 1   division   726 non-null    object
 2   district   726 non-null    object
 3   upazila    726 non-null    object
dtypes: object(4)
memory usage: 22.8+ KB


In [135]:
Fact_Table = pd.read_excel(open('e-commerece_data.xlsx','rb'), sheet_name=0, engine='openpyxl')
Fact_Table

,payment_key,coustomer_key,time_key,item_key,store_key,quantity,unit,unit_price,total_price
0,P026,C004510,T049189,I00177,S00307,1,ct,35.0,35.0
1,P022,C008967,T041209,I00248,S00595,1,rolls,26.0,26.0
2,P030,C007261,T03633,I00195,S00496,8,ct,12.5,100.0
3,P032,C007048,T084631,I00131,S0086,8,ct,14.0,112.0
4,P014,C006430,T071276,I00050,S00488,8,cans,8.0,64.0
...,...,...,...,...,...,...,...,...,...
999995,P010,C003726,T09247,I00063,S00502,5,cans,40.0,200.0
999996,P021,C002581,T023483,I00252,S00225,1,ct,11.0,11.0
999997,P035,C006133,T039372,I00119,S00555,7,ct,53.0,371.0
999998,P031,C008735,T097975,I00257,S00440,2,ct,21.0,42.0


In [138]:
Fact_Table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   payment_key    1000000 non-null  object 
 1   coustomer_key  1000000 non-null  object 
 2   time_key       1000000 non-null  object 
 3   item_key       1000000 non-null  object 
 4   store_key      1000000 non-null  object 
 5   quantity       1000000 non-null  int64  
 6   unit           996277 non-null   object 
 7   unit_price     1000000 non-null  float64
 8   total_price    1000000 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 68.7+ MB


In [139]:
Fact_Table.isnull().sum()

payment_key         0
coustomer_key       0
time_key            0
item_key            0
store_key           0
quantity            0
unit             3723
unit_price          0
total_price         0
dtype: int64

### Transform

***Required Data Pre-Processing***

In [153]:
for x in Item_dim.index:
    c=str(Item_dim.loc[x, "desc"])
    Item_dim.loc[x, "desc"] = c.replace("a.",'')
for x in Item_dim['desc']:
    print(x)

 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage - Soda
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Sparkling Water
 Beverage Spa

In [52]:
Time_dim['date'] = pd.to_datetime(Time_dim['date'], format='%d-%m-%Y %H:%M')

In [133]:
Time_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   time_key  99999 non-null  object        
 1   date      99999 non-null  datetime64[ns]
 2   hour      99999 non-null  int64         
 3   day       99999 non-null  int64         
 4   week      99999 non-null  object        
 5   month     99999 non-null  int64         
 6   quarter   99999 non-null  object        
 7   year      99999 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 6.1+ MB


### Load

***Connect to the database***

In [140]:
class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="ecomdb",
                                           user = "postgres",
                                           password = "postgres",
                                           host = "127.0.0.1",
                                           port = "5432")

    def getConnection(self):
        print("successfully connected to database")
        return self.connection

In [167]:
con = PostgresConnection().getConnection()

successfully connected to database


In [142]:
insert_stmt = "INSERT INTO ecomdb_star_schema.trans_dim ( payment_key, trans_type, bank_name)" \
            " VALUES (%s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, Trans_dim.values)
con.commit()
cur.close()

In [163]:
insert_stmt = "INSERT INTO ecomdb_star_schema.store_dim ( store_key, division, district,upazila)" \
            " VALUES (%s, %s, %s,%s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, Store_dim.values)
con.commit()
cur.close()

In [165]:
insert_stmt = "INSERT INTO ecomdb_star_schema.coustomer_dim ( coustomer_key, name, contact_no,nid)" \
            " VALUES (%s, %s, %d,%d)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, Coustomer_dim.values)
con.commit()
cur.close()

ValueError: unsupported format character 'd' (0x64) at index 101

In [160]:
insert_stmt = "INSERT INTO ecomdb_star_schema.item_dim ( item_key, item_name,desc,unit_price,man_country,supplier, unit)" \
            " VALUES (%s, %s, %s, %f, %s,%s,%s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, Item_dim.values)
con.commit()
cur.close()

ValueError: unsupported format character 'f' (0x66) at index 127

***Retrieving Data***

In [171]:
cur = con.cursor()
select_stmt = "SELECT t.payment_key , t.trans_type, t.bank_name " \
              "FROM ecomdb_star_schema.trans_dim t"
cur.execute(select_stmt)
records = cur.fetchall()
records

[('P001', 'cash', 'None'),
 ('P002', 'card', 'AB Bank Limited'),
 ('P003', 'card', 'Bangladesh Commerce Bank Limited'),
 ('P004', 'card', 'Bank Asia Limited'),
 ('P005', 'card', 'BRAC Bank Limited'),
 ('P006', 'card', 'Citizens Bank Limited'),
 ('P007', 'card', 'City Bank Limited'),
 ('P008', 'card', 'Community Bank Bangladesh Limited'),
 ('P009', 'card', 'Dhaka Bank Limited'),
 ('P010', 'card', 'Dutch-Bangla Bank Limited'),
 ('P011', 'card', 'Eastern Bank Limited'),
 ('P012', 'card', 'IFIC Bank Limited'),
 ('P013', 'card', 'Jamuna Bank Limited'),
 ('P014', 'card', 'Meghna Bank Limited'),
 ('P015', 'card', 'Mercantile Bank Limited'),
 ('P016', 'card', 'Midland Bank Limited'),
 ('P017', 'card', 'Modhumoti Bank Limited'),
 ('P018', 'card', 'Mutual Trust Bank Limited'),
 ('P019', 'card', 'National Bank Limited'),
 ('P020', 'card', 'National Credit & Commerce Bank Limited'),
 ('P021', 'card', 'NRB Bank Limited'),
 ('P022', 'card', 'NRB Commercial Bank Ltd'),
 ('P023', 'card', 'NRB Global B

In [172]:
trans_df = pd.DataFrame(list(records), columns=['payment_key', 'trans_type', 'bank_name'])
trans_df

,payment_key,trans_type,bank_name
0,P001,cash,None
1,P002,card,AB Bank Limited
2,P003,card,Bangladesh Commerce Bank Limited
3,P004,card,Bank Asia Limited
4,P005,card,BRAC Bank Limited
5,P006,card,Citizens Bank Limited
6,P007,card,City Bank Limited
7,P008,card,Community Bank Bangladesh Limited
8,P009,card,Dhaka Bank Limited
9,P010,card,Dutch-Bangla Bank Limited


In [169]:
cur = con.cursor()
select_stmt = "SELECT * FROM ecomdb_star_schema.store_dim"
cur.execute(select_stmt)
records = cur.fetchall()
records

[('S0001', 'SYLHET', 'HABIGANJ', 'AJMIRIGANJ'),
 ('S0002', 'SYLHET', 'HABIGANJ', 'BAHUBAL'),
 ('S0003', 'SYLHET', 'HABIGANJ', 'BANIACHONG'),
 ('S0004', 'SYLHET', 'HABIGANJ', 'CHUNARUGHAT'),
 ('S0005', 'SYLHET', 'HABIGANJ', 'HABIGANJ SADAR'),
 ('S0006', 'SYLHET', 'HABIGANJ', 'LAKHAI'),
 ('S0007', 'SYLHET', 'HABIGANJ', 'MADHABPUR'),
 ('S0008', 'SYLHET', 'HABIGANJ', 'NABIGANJ'),
 ('S0009', 'SYLHET', 'MAULVIBAZAR', 'BARLEKHA'),
 ('S0010', 'SYLHET', 'MAULVIBAZAR', 'JURI'),
 ('S0011', 'SYLHET', 'MAULVIBAZAR', 'KAMALGANJ'),
 ('S0012', 'SYLHET', 'MAULVIBAZAR', 'KULAURA'),
 ('S0013', 'SYLHET', 'MAULVIBAZAR', 'MAULVIBAZAR SADAR'),
 ('S0014', 'SYLHET', 'MAULVIBAZAR', 'RAJNAGAR'),
 ('S0015', 'SYLHET', 'MAULVIBAZAR', 'SREEMANGAL'),
 ('S0016', 'SYLHET', 'SUNAMGANJ', 'BISHWAMBARPUR'),
 ('S0017', 'SYLHET', 'SUNAMGANJ', 'CHHATAK'),
 ('S0018', 'SYLHET', 'SUNAMGANJ', 'DAKSHIN SUNAMGANJ'),
 ('S0019', 'SYLHET', 'SUNAMGANJ', 'DERAI'),
 ('S0020', 'SYLHET', 'SUNAMGANJ', 'DHARAMPASHA'),
 ('S0021', 'SYLHET', 'S

***creating tables on ecomdb***

In [7]:
def create_tables():
    """ create tables in the PostgreSQL database"""
    commands = (
        """ CREATE TABLE Trans_dim (
                payment_key varchar(50) NOT NULL,
                trans_type varchar(10) NOT NULL,
                bank_name varchar(100) NOT NULL,
                CONSTRAINT PK_7 PRIMARY KEY ( payment_key )
        )
        """,
        """ CREATE TABLE Coustomer_dim (
                coustomer_key varchar(50) NOT NULL,
                name varchar(50) NULL,
                contact_no varchar(20) NOT NULL,
                nid integer NOT NULL,
                CONSTRAINT PK_21 PRIMARY KEY ( coustomer_key )
        )
        """,
        
        """ CREATE TABLE Time_dim (
                time_key varchar(50) NOT NULL,
                "date" timestamp NOT NULL,
                hour integer NOT NULL,
                day integer NOT NULL,
                week varchar(50) NOT NULL,
                month integer NOT NULL,
                quarter varchar(10) NOT NULL,
                year integer NOT NULL,
                CONSTRAINT PK_27 PRIMARY KEY ( time_key )
        )
        """,
        
        """ CREATE TABLE Item_dim (
                item_key varchar(50) NOT NULL,
                item_name varchar(100) NOT NULL,
                "desc" varchar(50) NOT NULL,
                unit_price numeric NOT NULL,
                man_country varchar(20) NOT NULL,
                supplier varchar(100) NOT NULL,
                unit varchar(10) NULL,
                CONSTRAINT PK_12 PRIMARY KEY ( item_key )
        )
        """,
        
        """ CREATE TABLE Store_dim (
                store_key varchar(50) NOT NULL,
                division  varchar(50) NOT NULL,
                district  varchar(50) NOT NULL,
                upazila   varchar(50) NOT NULL,
                CONSTRAINT PK_37 PRIMARY KEY ( store_key )
        )
        """,
        """ CREATE TABLE Fact_table (
                payment_key   varchar(50) NOT NULL,
                store_key     varchar(50) NOT NULL,
                item_key      varchar(50) NOT NULL,
                time_key      varchar(50) NOT NULL,
                coustomer_key varchar(50) NOT NULL,
                quantity      integer NOT NULL,
                unit          varchar(50) NULL,
                unit_price    numeric NOT NULL,
                total_price   numeric NOT NULL,
                CONSTRAINT FK_46 FOREIGN KEY ( payment_key ) REFERENCES Trans_dim ( payment_key ),
                CONSTRAINT FK_49 FOREIGN KEY ( coustomer_key ) REFERENCES Coustomer_dim ( coustomer_key ),
                CONSTRAINT FK_54 FOREIGN KEY ( time_key ) REFERENCES Time_dim ( time_key ),
                CONSTRAINT FK_57 FOREIGN KEY ( item_key ) REFERENCES Item_dim ( item_key ),
                CONSTRAINT FK_60 FOREIGN KEY ( store_key ) REFERENCES Store_dim ( store_key )
        )
        """,
        """
        CREATE INDEX FK_48 ON Fact_table (
             payment_key)
        """,
        """CREATE INDEX FK_51 ON Fact_table( 
             coustomer_key
        )
        """, 
        """CREATE INDEX FK_56 ON Fact_table(
             time_key
        )
        """, 
        """CREATE INDEX FK_59 ON Fact_table(
            item_key
        )
        """,
        """
        CREATE INDEX FK_62 ON Fact_table (
            store_key
        )
        """
        )
    try:
        cur = con.cursor()
        # create table one by one
        for command in commands:
            cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if con is not None:
            con.close()


if __name__ == '__main__':
    create_tables()